In [157]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies



In [158]:

query = db.comp.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1,
   "category_code":1,
   "founded_year":1,
   "products":1,
   "relationships":1,
   "competitions":1,
   "total_money_raised":1,
   "description":1,
   "offices":1})



In [159]:
import pandas as pd
df2=pd.DataFrame(query)
df2.head()


,_id,category_code,competitions,description,founded_year,name,offices,products,relationships,total_money_raised
0,52cdef7c4bab8bd675297d8b,enterprise,[],Server Management Software,1996.0,AdventNet,"[{'description': 'Headquarters', 'address1': '...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",$0
1,52cdef7c4bab8bd675297d8a,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",Technology Platform Company,2005.0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...",$39.8M
2,52cdef7c4bab8bd675297d8c,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Online Business Apps Suite,2005.0,Zoho,"[{'description': 'Headquarters', 'address1': '...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...",$0
3,52cdef7c4bab8bd675297d8d,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",user driven social content website,2004.0,Digg,"[{'description': None, 'address1': '135 Missis...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ...",$45M
4,52cdef7c4bab8bd675297d8f,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",None,2005.0,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per...",$800k


In [160]:
df=df2.copy() #en caso de necesitar volver al original no será 
#necesario ejecutal la query

In [161]:
#LIMPIAMOS LA COLUMNA TOTAL_MONEY_RAISED
import re
def modStr(s,caracter):
    return (s[:s.find(caracter)]+s[s.find(caracter)+1:])
def todoDolares(x):
    d={
        "M":1000000,
        "B":1000000000,
        "k":1000,
    }
    s=str(x) 
    r=0
    for i,e in d.items():
        if i in s: 
            s = modStr(s,i)
            if "£" in s: r=float(s[s.find("£")+1:])*1.25*e
            elif "€" in s: r=float(s[s.find("€")+1:])*1.12*e
            elif "$" in s: r=float(s[s.find("$")+1:])*e
    try:  return int(r)
    except ValueError: return int(re.sub(r'\W','',re.sub(r'[a-zA-Z]','',s)))

money=df["total_money_raised"]
money=money.apply(todoDolares)
print(money.value_counts().head())
print(df["total_money_raised"].value_counts().head())


0           9173
1000000       85
10000000      85
5000000       79
2000000       73
Name: total_money_raised, dtype: int64
$0      9168
$10M      85
$1M       83
$5M       75
$2M       73
Name: total_money_raised, dtype: int64


In [162]:
df["total_money_raised"]=money
df.head()

,_id,category_code,competitions,description,founded_year,name,offices,products,relationships,total_money_raised
0,52cdef7c4bab8bd675297d8b,enterprise,[],Server Management Software,1996.0,AdventNet,"[{'description': 'Headquarters', 'address1': '...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",0
1,52cdef7c4bab8bd675297d8a,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",Technology Platform Company,2005.0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...",39800000
2,52cdef7c4bab8bd675297d8c,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Online Business Apps Suite,2005.0,Zoho,"[{'description': 'Headquarters', 'address1': '...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...",0
3,52cdef7c4bab8bd675297d8d,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",user driven social content website,2004.0,Digg,"[{'description': None, 'address1': '135 Missis...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ...",45000000
4,52cdef7c4bab8bd675297d8f,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",None,2005.0,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per...",800000


In [163]:
years=df["founded_year"]
years.fillna("0",inplace=True)
years=years.apply(lambda x: int(x))
years.value_counts().head()

0       2487
2007    1947
2008    1795
2006    1286
2005     870
Name: founded_year, dtype: int64

In [164]:
def cantidadTiempo(x):
    if int(x)!=0: return (2019-int(x))
    else: return "No se sabe"
    
df["founded_year"]=years
df.head()

,_id,category_code,competitions,description,founded_year,name,offices,products,relationships,total_money_raised
0,52cdef7c4bab8bd675297d8b,enterprise,[],Server Management Software,1996,AdventNet,"[{'description': 'Headquarters', 'address1': '...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",0
1,52cdef7c4bab8bd675297d8a,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",Technology Platform Company,2005,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...",39800000
2,52cdef7c4bab8bd675297d8c,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Online Business Apps Suite,2005,Zoho,"[{'description': 'Headquarters', 'address1': '...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...",0
3,52cdef7c4bab8bd675297d8d,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",user driven social content website,2004,Digg,"[{'description': None, 'address1': '135 Missis...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ...",45000000
4,52cdef7c4bab8bd675297d8f,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",None,2005,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per...",800000


In [165]:
new_col=df[["competitions","offices","products","relationships"]]
new_col.head()

,competitions,offices,products,relationships
0,[],"[{'description': 'Headquarters', 'address1': '...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde..."
1,"[{'competitor': {'name': 'Wikia', 'permalink':...","[{'description': '', 'address1': '710 - 2nd Av...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V..."
2,"[{'competitor': {'name': 'Empressr', 'permalin...","[{'description': 'Headquarters', 'address1': '...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'..."
3,"[{'competitor': {'name': 'Reddit', 'permalink'...","[{'description': None, 'address1': '135 Missis...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ..."
4,"[{'competitor': {'name': 'Dropbox', 'permalink...","[{'description': '', 'address1': 'Suite 200', ...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per..."


In [166]:
def nuevasColumnas(x):
    #COLUMNA DE COMPETITIONS
    comp=x["competitions"]
    lst_c=[]
    if comp!=[]:
        for e in comp:
            lst_c.append(e["competitor"]["name"])
    #COLUMNA DE OFFICES
    off=x["offices"]
    lst_o=[]
    lat=[]
    lon=[]
    principal=None
    for e in off:
        lat.append(e["latitude"])
        lon.append(e["longitude"])
        if e["latitude"] and e["longitude"]:
            principal={
                "type":"Point",
                "coordinates":[e["longitude"],e["latitude"]]    
            }
        lst_o.append(principal)
    tot=len(off) #Guardamos el tamaño de todas las oficinas de la empresa
    #COLUMNA DE RELATIONSHIPS
    rel=x["relationships"]
    lst_r=[]
    for e in rel:
        if not e["is_past"]:
            lst_r.append(e["title"])
    #COLUMNA DE PRODUCTOS
    prod=x["products"]
    lst_p=[]
    for e in prod:
        lst_p.append(e["name"])
    return {
       "totalOffices": tot,
        "Offices": lst_o,
        "Latitude": lat,
        "Longitude":lon,
        "totalCompetitors": len(comp),
        "Competitors": lst_c,
        "totalRelationships": len(rel),
        "Relationships": lst_r,
        "totalProducts":len(prod),
        "Products": lst_p      
    }


hola=new_col.apply(nuevasColumnas,result_type="expand",axis=1)
hola.head()

,Competitors,Latitude,Longitude,Offices,Products,Relationships,totalCompetitors,totalOffices,totalProducts,totalRelationships
0,[],[37.692934],[-121.904945],"[{'type': 'Point', 'coordinates': [-121.904945...",[],[],0,1,0,7
1,"[Wikia, JotSpot, Socialtext, Ning by Glam Medi...","[47.603122, 40.7237306]","[-122.333253, -73.9964312]","[{'type': 'Point', 'coordinates': [-122.333253...","[Wikison Wetpaint, Wetpaint Social Distributio...","[Co-Founder and VP, Social and Audience Develo...",8,2,2,17
2,"[Empressr, Transmedia Corporation, Live Docume...",[37.692934],[-121.904945],"[{'type': 'Point', 'coordinates': [-121.904945...","[Zoho Office Suite, Zoho CRM, Zoho Assist]","[CEO and Founder, VP Product Managment, VP Mar...",26,1,3,18
3,"[Reddit, Twitter, Newsvine, Truemors, fav.or.i...",[37.764726],[-122.394523],"[{'type': 'Point', 'coordinates': [-122.394523...",[Digg],"[CEO, CEO, CFO, VP of Engineering, Designer, N...",28,1,1,58
4,"[Dropbox, filobite, digitalbucket.net, Elephan...",[None],[None],[None],[Omnidrive],[],5,1,1,4


In [168]:
df_new=pd.concat([df,hola],axis=1)[["name","Offices","Latitude","Longitude"]]
df_new.head()

,name,Offices,Latitude,Longitude
0,AdventNet,"[{'type': 'Point', 'coordinates': [-121.904945...",[37.692934],[-121.904945]
1,Wetpaint,"[{'type': 'Point', 'coordinates': [-122.333253...","[47.603122, 40.7237306]","[-122.333253, -73.9964312]"
2,Zoho,"[{'type': 'Point', 'coordinates': [-121.904945...",[37.692934],[-121.904945]
3,Digg,"[{'type': 'Point', 'coordinates': [-122.394523...",[37.764726],[-122.394523]
4,Omnidrive,[None],[None],[None]


In [72]:
d=dict()
i=0
for e in list(df_new["name"].values):
    d[e]= list(df_new["Offices"].values)[i]
    i+=1

        

In [73]:
#67
d_new={
    "Name": list(d.keys())
}
for i in range(67):
    lst=[]
    for e in list(d.values()):
        try:
            lst.append(e[i])
        except IndexError:
            lst.append(None)
    d_new["Office_{}".format(i+1)]=lst


    


In [74]:
df_oficinas = pd.DataFrame(data=d_new)
display(df_oficinas.head())
df_oficinas[df_oficinas["Name"]=="Google"]["Office_67"]

,Name,Office_1,Office_2,Office_3,Office_4,Office_5,Office_6,Office_7,Office_8,Office_9,...,Office_58,Office_59,Office_60,Office_61,Office_62,Office_63,Office_64,Office_65,Office_66,Office_67
0,AdventNet,"{'type': 'Point', 'coordinates': [-121.904945,...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,...","{'type': 'Point', 'coordinates': [-73.9964312,...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Digg,"{'type': 'Point', 'coordinates': [-122.394523,...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Omnidrive,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


99    {'type': 'Point', 'coordinates': [-99.030156, ...
Name: Office_67, dtype: object

In [75]:
df_new2=pd.concat([df,hola],axis=1)[["name","Products","totalProducts"]]
df_new2.head()

,name,Products,totalProducts
0,AdventNet,[],0
1,Wetpaint,"[Wikison Wetpaint, Wetpaint Social Distributio...",2
2,Zoho,"[Zoho Office Suite, Zoho CRM, Zoho Assist]",3
3,Digg,[Digg],1
4,Omnidrive,[Omnidrive],1


In [76]:
def createDict(df,buscar,maximo,columna):
    d_sol={
        "Name": list(df["name"].values)
        }

    for i in range(maximo):
        lst=[]
        for e in list(df[columna].values):
            try:
                lst.append(e[i])
            except IndexError:
                lst.append(None)
        d_sol["{}_{}".format(buscar,i+1)]=lst
    return d_sol

d_prod=createDict(df_new2,"Product",df_new2["totalProducts"].max(),"Products")
df_prod=pd.DataFrame(data=d_prod)
df_prod.head()

,Name,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,Product_7,Product_8,Product_9,...,Product_143,Product_144,Product_145,Product_146,Product_147,Product_148,Product_149,Product_150,Product_151,Product_152
0,AdventNet,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Wetpaint,Wikison Wetpaint,Wetpaint Social Distribution System,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Zoho,Zoho Office Suite,Zoho CRM,Zoho Assist,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Digg,Digg,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Omnidrive,Omnidrive,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [77]:
dd=dict()
names=[]
off=[]
for n,o in d.items():
    for e in o:
        names.append(n)
        off.append(e)
dd={
    "Names": names,
    "Offices": off
}
dff=pd.DataFrame(data=dd)  
dff.head()

,Names,Offices
0,AdventNet,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,Wetpaint,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,Digg,"{'type': 'Point', 'coordinates': [-122.394523,..."


In [173]:
def latitudeLongitude(x):
    data=x["Offices"]
    if data:
        lat=  data.get("coordinates")[0]
        lon= data.get("coordinates")[1]
    else:
        lat=None
        lon=None
    return {
        "Latitude": lat,
        "Longitude": lon
    }
df_lon_lat=dff.apply(latitudeLongitude,result_type="expand",axis=1)
df_lon_lat.head()
dff=pd.concat([dff,df_lon_lat],axis=1)
dff.head()

,Names,Offices,Latitude,Longitude
0,AdventNet,"{'type': 'Point', 'coordinates': [-121.904945,...",-121.904945,37.692934
1,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,...",-122.333253,47.603122
2,Wetpaint,"{'type': 'Point', 'coordinates': [-73.9964312,...",-73.996431,40.723731
3,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,...",-121.904945,37.692934
4,Digg,"{'type': 'Point', 'coordinates': [-122.394523,...",-122.394523,37.764726


In [78]:
import os
from dotenv import load_dotenv
load_dotenv()
BASE_URL="https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="
buscar="Starbucks"
key="&key="+ os.environ["KEY"]

In [79]:
import requests
#res=requests.get(BASE_URL+buscar+key)


In [80]:
#res.json()

In [148]:
years=df[["founded_year","name"]]
def tiempos(x):
    try:
        if (2019-int(x["founded_year"]))>10 and int(x["founded_year"])>1000:
            comp=x["name"]
            tiempo=(2019-int(x["founded_year"]))
        else:
            comp=None
            tiempo=None
    except ValueError:
        comp=None
        tiempo=None
    return {
        "Comp_open_10": comp,
        "Years_open": tiempo
    }

new_col=years.apply(tiempos,result_type="expand",axis=1)
new_col.head()


,Comp_open_10,Years_open
0,AdventNet,23.0
1,Wetpaint,14.0
2,Zoho,14.0
3,Digg,15.0
4,Omnidrive,14.0


In [149]:
df_more_10=pd.concat([df,new_col],axis=1)[["name","Comp_open_10","Years_open"]]
df_more_10.head()

,name,Comp_open_10,Years_open
0,AdventNet,AdventNet,23.0
1,Wetpaint,Wetpaint,14.0
2,Zoho,Zoho,14.0
3,Digg,Digg,15.0
4,Omnidrive,Omnidrive,14.0


In [150]:
comp_mas_10=list(new_col[new_col["Comp_open_10"]!=None]["Comp_open_10"].values)
print(len(set(comp_mas_10)))
print(dff.shape)

10193
(15950, 2)


In [174]:
def posiciondeComp(x):
    if x["Names"] in comp_mas_10:
        name=x["Names"]
        off=x["Offices"]
        lat=x["Latitude"]
        lon=x["Longitude"]
    else:
        name=None
        off=None
        lat=None
        lon=None
    return {
        "Name": name,
        "Offices": off,
        "Latitude":lat,
        "Longitude":lon
    }

df_dont_want=dff.apply(posiciondeComp,result_type="expand",axis=1).dropna()

df_dont_want.shape

(9280, 4)

In [ ]:
"""def findNear(geopoint, radio_max_meters=2000):
    return db.first_office.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}

num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))
"""

In [85]:
#CARGAMOS UN CSV CON LOS AEROPUERTOS
import urllib3
import csv

URLAE="https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"

#response=urllib3.urlopen(URLAE)
#cr=csv.reader(response)
#http=urllib3.PoolManager()
#resp=http.request("GET",url)

#data=resp.data.decode("UTF-8")
#data

AttributeError: module 'urllib3' has no attribute 'urlopen'

In [88]:
from urllib.request import urlopen
import ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context
r=urlopen(URLAE)

In [90]:
with open("Aeropuertos.csv","wb") as f:
    f.write(r.read())
r.close()

In [94]:
names=["Airport ID","Name","City","Country","IATA","ICAO",
      "Latitude","Longitude","Altitude","Timezone","DST","Tz",
      "Type","Source"]

df_ae=pd.read_csv("Aeropuertos.csv",names=names)
df_ae.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [96]:
df_ae2=df_ae[["Name","City","Latitude","Longitude"]]
df_ae2.head() 

,Name,City,Latitude,Longitude
0,Goroka Airport,Goroka,-6.081690,145.391998
1,Madang Airport,Madang,-5.207080,145.789001
2,Mount Hagen Kagamuga Airport,Mount Hagen,-5.826790,144.296005
3,Nadzab Airport,Nadzab,-6.569803,146.725977
4,Port Moresby Jacksons International Airport,Port Moresby,-9.443380,147.220001


In [98]:
def geoloc(x):
    principal=None
    if x["Latitude"] and x["Longitude"]:
        principal={
            "type":"Point",
            "coordinates":[x["Longitude"],x["Latitude"]]    
            }
    return {
        "Localizacion": principal
    }
df_loc=df_ae2.apply(geoloc,result_type="expand",axis=1)
df_loc=pd.concat([df_ae2,df_loc], axis=1)
df_loc.head()

,Name,City,Latitude,Longitude,Localizacion
0,Goroka Airport,Goroka,-6.081690,145.391998,"{'type': 'Point', 'coordinates': [145.39199829..."
1,Madang Airport,Madang,-5.207080,145.789001,"{'type': 'Point', 'coordinates': [145.78900146..."
2,Mount Hagen Kagamuga Airport,Mount Hagen,-5.826790,144.296005,"{'type': 'Point', 'coordinates': [144.29600524..."
3,Nadzab Airport,Nadzab,-6.569803,146.725977,"{'type': 'Point', 'coordinates': [146.725977, ..."
4,Port Moresby Jacksons International Airport,Port Moresby,-9.443380,147.220001,"{'type': 'Point', 'coordinates': [147.22000122..."


In [104]:
ddprod=dict()
names=[]
prod=[]
d_product=dict()
i=0
for e in list(df_new2["name"].values):
    d_product[e]= list(df_new2["Products"].values)[i]
    i+=1



print(list(d_product.items())[0])
for n,p in list(d_product.items()):
    for e in p:
        names.append(n)
        prod.append(e)
ddprod={
    "Names": names,
    "Productos": prod
}
dff_prod=pd.DataFrame(data=ddprod)  
dff_prod.head()

('AdventNet', [])


,Names,Productos
0,Wetpaint,Wikison Wetpaint
1,Wetpaint,Wetpaint Social Distribution System
2,Zoho,Zoho Office Suite
3,Zoho,Zoho CRM
4,Zoho,Zoho Assist


In [176]:
df_design=df[(df["category_code"]=="design") | (df["category_code"]=="fashion")]
def posDesign(x):
    if x["Names"] in list(df_design["name"].values):
        name=x["Names"]
        off=x["Offices"]
        lat=x["Latitude"]
        lon=x["Longitude"]
    else:
        name=None
        off=None
        lon=None
        lat=None

    return {
        "Name": name,
        "Offices": off,
        "Latitude":lat,
        "Longitude":lon
    }
df_d=dff.apply(posDesign,result_type="expand",axis=1).dropna()
df_d.shape


(9, 4)

In [177]:
df_d

,Latitude,Longitude,Name,Offices
1254,-73.980064,40.747270,Gilt Groupe,"{'type': 'Point', 'coordinates': [-73.9800645,..."
2022,-122.400598,37.795531,99designs,"{'type': 'Point', 'coordinates': [-122.4005983..."
2023,144.986855,-37.802659,99designs,"{'type': 'Point', 'coordinates': [144.9868546,..."
2024,13.446903,52.498620,99designs,"{'type': 'Point', 'coordinates': [13.4469031, ..."
6692,-74.000827,40.730763,Stylesight,"{'type': 'Point', 'coordinates': [-74.000827, ..."
8771,-73.950821,40.727719,Refinery29,"{'type': 'Point', 'coordinates': [-73.9508211,..."
10229,-74.075833,4.598056,Geelbe,"{'type': 'Point', 'coordinates': [-74.0758333,..."
13171,-122.693144,45.796841,Graticle,"{'type': 'Point', 'coordinates': [-122.6931439..."
13933,-2.930832,43.264499,Chicisimo,"{'type': 'Point', 'coordinates': [-2.9308322, ..."


In [178]:
df_1M=df[df["total_money_raised"]>=1000000]
def more1M(x):
    if x["Names"] in list(df_1M["name"].values):
        name=x["Names"]
        off=x["Offices"]
        lat=x["Latitude"]
        lon=x["Longitude"]
    else:
        name=None
        off=None
        lat=None
        lon=None

    return {
        "Name": name,
        "Offices": off,
        "Latitude":lat,
        "Longitude":lon
    }
df_1M=dff.apply(more1M,result_type="expand",axis=1).dropna()
df_1M.shape


(3808, 4)

In [179]:
df_educ=df[df["category_code"]=="education"]
def educationOffices(x):
    if x["Names"] in list(df_educ["name"].values):
        name=x["Names"]
        off=x["Offices"]
        lat=x["Latitude"]
        lon=x["Longitude"]
    else:
        name=None
        off=None
        lat=None
        lon=None

    return {
        "Name": name,
        "Offices": off,
        "Latitude":lat,
        "Longitude":lon
    }
df_educ=dff.apply(educationOffices,result_type="expand",axis=1).dropna()
df_educ.shape

(27, 4)

In [181]:
#IMPORTAMOS:

#JSON-> AEROPUERTOS 
df_loc.to_json("AeropuertosLoc.json",default_handler=str,orient="records")
#JSON-> COLEGIOS
df_educ.to_json("Colegios.json",default_handler=str,orient="records")
#JSON-> COMPAÑIAS CON MAS DE 10 AÑOS 
df_dont_want.to_json("CompMore10Years.json",default_handler=str,orient="records")
#JSON-> COMPAÑIAS DE DISEÑO
df_d.to_json("Design.json",default_handler=str,orient="records")
#JSON-> COMPAÑIAS QUE EMPEZARON CON MAS DE $1M
df_1M.to_json("Comp1M.json",default_handler=str,orient="records")